## Tutorial 2 : Build Graph Neural Networks with PyG

In this tutorial, we will learn how to build a graph neural network with PyG.
PyG offers various handy features when it comes to build GNNs, including
- An extended `Sequential` module that can be used to build GNN
- Pre-implemented and, also, optimized graph convolutional layers
- Graph Neural Network implementations

### A limitation of `torch.nn.Sequential`

In native PyTorch, `torch.nn.Sequential` is a handy module that allows us to build a neural network in a sequential manner. For example, we can build a simple MLP with `torch.nn.Sequential` as follows:

```python
mlp = nn.Sequential(
    nn.Linear(32, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
)
```

`Sequential` class minimizes the boiler plate code for implementing `forward` methods. 
We can implement equivalent MLP without using `Sequential` as follows:

```python
import torch.nn as nn

class MLP(nn.Module):
    
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList([nn.Linear(32, 32), 
                                     nn.ReLU(), 
                                     nn.Linear(32, 1)])
    
    def forward(self, x):        
        for layer in self.layers:
            x = layer(x)
        return x

mlp = MLP()
```



However, `torch.nn.Sequential` has a limitation that each layer should have only one input and output.
This limitation becomes a problem when it comes to building a graph neural network. For instance, when `INLayer` takes
two inputs, node and edge features and return two outputs updated node and edge features. Hence
it is less trivial to build a graph neural network with `torch.nn.Sequential`.

### Using `pytorch_geometric.nn.Sequential` to build GNN

`pytorch_geometric.nn.Sequential` is an extended version of `torch.nn.Sequential` that allows us to build a graph neural network in a sequential manner. Let's see how we can build a graph neural network with `pytorch_geometric.nn.Sequential`.

In [9]:
from torch_geometric.nn import Sequential
from torch_geometric.data import Batch

from common.graph_gen import generate_random_graph
from common.layers import InteractionNetworkLayer

print(help(InteractionNetworkLayer.forward))

Help on function forward in module common.layers:

forward(self, x, edge_index, edge_attr) -> Tuple[torch.Tensor, torch.Tensor]
    Runs the forward pass of the module.

None


In [10]:
dim = 5
model = Sequential("x, edge_index, edge_attr", # input
                   [
                       (InteractionNetworkLayer(dim), "x, edge_index, edge_attr -> x, edge_attr"), 
                       (InteractionNetworkLayer(dim), "x, edge_index, edge_attr -> x, edge_attr"),
                   ]
)

In [11]:
gs = Batch.from_data_list([generate_random_graph(10,
                                                 node_feat_dim=dim,
                                                 edge_feat_dim=dim) for _ in range(2)])

In [15]:
unf, uef = model(gs.x, gs.edge_index, gs.edge_attr) # update node feature (unf), updated edge feature (uef)